In [1]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver


                                              0.0/6.6 MB ? eta -:--:--
                                              0.1/6.6 MB 6.1 MB/s eta 0:00:02
     -                                        0.2/6.6 MB 2.9 MB/s eta 0:00:03
     --                                       0.4/6.6 MB 2.7 MB/s eta 0:00:03
     ---                                      0.6/6.6 MB 2.9 MB/s eta 0:00:03
     ----                                     0.7/6.6 MB 2.9 MB/s eta 0:00:02
     -----                                    0.8/6.6 MB 3.1 MB/s eta 0:00:02
     ------                                   1.0/6.6 MB 3.2 MB/s eta 0:00:02
     -------                                  1.2/6.6 MB 3.2 MB/s eta 0:00:02
     --------                                 1.3/6.6 MB 3.1 MB/s eta 0:00:02
     --------                                 1.4/6.6 MB 3.0 MB/s eta 0:00:02
     ---------                                1.6/6.6 MB 3.0 MB/s eta 0:00:02
     ---------                                1.6/6.6 MB 2.9 MB/s eta 0

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [4]:
# Global driver to use throughout the script
DRIVER = None

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if force_restart:
        close_driver()
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

In [5]:
# Urls
TIKI = 'https://tiki.vn'

In [6]:
#Set up categories and links
#@title <a name="tiki-cats"></a> { form-width:'1px' }
# Pre-defined links to each categories
MAIN_CATEGORIES = [
    {'Name': 'Đồ Chơi - Mẹ &amp; Bé',            'URL': 'https://tiki.vn/do-choi-me-be/c2549'},
    {'Name': 'Điện Thoại - Máy Tính Bảng',       'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789'},
    {'Name': 'Làm Đẹp - Sức Khỏe',               'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520'},
    {'Name': 'Điện Gia Dụng',                    'URL': 'https://tiki.vn/dien-gia-dung/c1882'},
    {'Name': 'Thời trang nữ',                    'URL': 'https://tiki.vn/thoi-trang-nu/c931'},
    {'Name': 'Thời trang nam',                   'URL': 'https://tiki.vn/thoi-trang-nam/c915'},
    {'Name': 'Giày - Dép nữ',                    'URL': 'https://tiki.vn/giay-dep-nu/c1703'},
    {'Name': 'Giày - Dép nam',                   'URL': 'https://tiki.vn/giay-dep-nam/c1686'},
    {'Name': 'Túi thời trang nữ',                'URL': 'https://tiki.vn/tui-vi-nu/c976'},
    {'Name': 'Túi thời trang nam',               'URL': 'https://tiki.vn/tui-thoi-trang-nam/c27616'},
    {'Name': 'Balo và Vali',                     'URL': 'https://tiki.vn/balo-va-vali/c6000'},
    {'Name': 'Phụ kiện thời trang',              'URL': 'https://tiki.vn/phu-kien-thoi-trang/c27498'},
    {'Name': 'Đồng hồ và Trang sức',             'URL': 'https://tiki.vn/dong-ho-va-trang-suc/c8371'},
    {'Name': 'Laptop - Máy Vi Tính - Linh kiện', 'URL': 'https://tiki.vn/laptop-may-vi-tinh-linh-kien/c1846'},
    {'Name': 'Nhà Cửa - Đời Sống',               'URL': 'https://tiki.vn/nha-cua-doi-song/c1883'},
    {'Name': 'Bách Hóa Online',                  'URL': 'https://tiki.vn/bach-hoa-online/c4384'},
    {'Name': 'Hàng Quốc Tế',                     'URL': 'https://tiki.vn/hang-quoc-te/c17166'},
    {'Name': 'Thiết Bị Số - Phụ Kiện Số',        'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815'},
    {'Name': 'Voucher - Dịch vụ',                'URL': 'https://tiki.vn/voucher-dich-vu/c11312'},
    {'Name': 'Ô Tô - Xe Máy - Xe Đạp',           'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594'},
    {'Name': 'Nhà Sách Tiki',                    'URL': 'https://tiki.vn/nha-sach-tiki/c8322'},
    {'Name': 'Điện Tử - Điện Lạnh',              'URL': 'https://tiki.vn/dien-tu-dien-lanh/c4221'},
    {'Name': 'Thể Thao - Dã Ngoại',              'URL': 'https://tiki.vn/the-thao-da-ngoai/c1975'},
    {'Name': 'Máy Ảnh - Máy Quay Phim',          'URL': 'https://tiki.vn/may-anh/c1801'}
]


In [7]:
#Function to get info from one product
close_driver()
start_driver()
DRIVER.get(MAIN_CATEGORIES[3]['URL']) 
time.sleep(3)  
DRIVER.current_url
all_products = DRIVER.find_elements(By.CLASS_NAME, "product-item")
len(all_products)

# Function to extract product info from the product
def get_product_info_single(product_item):
  info = {'name': '', 'price': '', 'product_url': '', 'image': ''} 

  # get name
  try:
    name_class = product_item.find_element(By.CLASS_NAME, 'name')
    info['name'] = name_class.find_element(By.TAG_NAME, 'h3').get_attribute('innerHTML')
  except NoSuchElementException:
    pass

  # get price
  try:
    info['price'] = product_item.find_element(By.CLASS_NAME, 'price-discount__price').get_attribute('innerHTML')
  except NoSuchElementException:
    print(info['price'])
    info['price'] = None
    
  # get product link
  try:
    info['product_url'] = product_item.get_attribute('href')
  except Exception:
    pass

  # get thumbnail url
  try:
    thumbnail = product_item.find_element(By.TAG_NAME, 'picture')
    info['image'] = thumbnail.find_element(By.TAG_NAME, 'img').get_attribute('src')
  except Exception:
    pass
  
  #Rating
  try:
    Rating = product_item.find_element(By.CLASS_NAME,'average').get_attribute('style')
    info['Rating'] = Rating[-5:-1]
  except Exception:
    info['Rating'] = None
  
  #Price discount
  try:
    info['discount'] = product_item.find_element(By.CLASS_NAME, 'price-discount__discount').get_attribute('innerHTML')
  except Exception:
    info['discount'] = None
  
  #Installment:
  try:
    info['installment']= product_item.find_element_by_css_selector('div.badge-benefits > div.item > span').get_attribute('innerHTML')
  except Exception:
    info['installment'] = None
  if info['installment'] != None:
    info['installment'] = True
  else:
    info['installment'] = False

  #Free gifts:
  try: 
     info['free_gifts'] = product_item.find_element_by_css_selector('div.freegift-list > span').get_attribute('innerHTML')
  except Exception:
    info['free_gifts'] = None
  if info['free_gifts'] != None:
    info['free_gifts'] = True
  else:
    info['free_gifts'] = False
  
  #TikiFast/Freeship:
  try:
    thumbnail1 = product_item.find_element(By. CLASS_NAME, "thumbnail")
    info['TikiFast'] = thumbnail1.find_element(By. TAG_NAME, 'img').get_attribute('src')
  except Exception:
    info['TikiFast'] = None
  if info['TikiFast'] == None:
    info['TikiFast'] = False
  else:
    info['TikiFast'] = True

  #TikiNow:
  try:
    thumbnail2 = product_item.find_element(By.CLASS_NAME, 'item')
    info['TikiNow'] = thumbnail2.find_element(By. TAG_NAME, 'img').get_attribute('src')
  except Exception:
    info['TikiNow'] = None
  if info['TikiNow'] == None:
    info['TikiNow'] = False
  else:
    info['TikiNow'] = True

  return info

In [8]:
#Function to scrap info of all products from a page URL
from selenium.common.exceptions import StaleElementReferenceException
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
  global DRIVER

  # YOUR CODE HERE

  data = []            # Store the info dictionary of each product in this list
                         # Use the driver to get info from the product page
  DRIVER.get(page_url)
  DRIVER.current_url
    
  time.sleep(3)        # Sleep AFTER loading website in order to wait for it to finish

  # Get a list of product elements. Print number of products found if desired
  all_products = DRIVER.find_elements(By.CLASS_NAME, 'product-item')
  print(len(all_products))
  
                # Loop through list of product elements, read and add each product info into `data`
  for i in all_products:
    info_item = get_product_info_single(i)
    data.append(info_item)
  return data

In [9]:
#Scrapping
cat_idx  = 3
main_cat = MAIN_CATEGORIES[cat_idx]

start_driver(force_restart=True)
print('Scraping', main_cat['Name'])
print('Link:', main_cat['URL'])

prod_data = [] # STORE YOUR PRODUCT INFO DICTIONARIES IN HERE


Scraping Điện Gia Dụng
Link: https://tiki.vn/dien-gia-dung/c1882


In [ ]:
### YOUR CODE HERE TO GET DATA ###
page = 3  
for num in range(page + 1):
  page_url = main_cat['URL']+'?page='+str(num)
  data = get_product_info_from_page(page_url)
  prod_data.extend(data)

close_driver() # Close driver when we're done



In [16]:
df = pd.DataFrame(data=prod_data, columns=prod_data[0].keys())
df.head()

,name,price,product_url,image,Rating,discount,installment,free_gifts,TikiFast,TikiNow
0,Bếp Từ Đơn Sunhouse SHD6803 (2000W) - Kèm Nồ...,829.000<sup> ₫</sup>,https://tka.tiki.vn/pixel/pixel?data=HqEkvFYs-...,https://salt.tikicdn.com/cache/280x280/media/c...,None,-<!-- -->30<!-- -->%,False,False,True,False
1,Máy Hút Bụi Cầm Tay Không Dây Damas XC628 - Ch...,459.000<sup> ₫</sup>,https://tka.tiki.vn/pixel/pixel?data=pOmk3iayb...,https://salt.tikicdn.com/cache/280x280/ts/prod...,None,-<!-- -->49<!-- -->%,False,False,True,False
2,Nồi cơm điện Cuckoo 1.8L CR-1005 lòng nồi chốn...,740.000<sup> ₫</sup>,https://tka.tiki.vn/pixel/pixel?data=r0xW5oAk_...,https://salt.tikicdn.com/cache/280x280/ts/prod...,None,-<!-- -->25<!-- -->%,False,False,True,False
3,Máy Xay Thịt 4 Lưỡi - Dung Tích 2 Lít - Công S...,149.000<sup> ₫</sup>,https://tka.tiki.vn/pixel/pixel?data=3tzLSBYIn...,https://salt.tikicdn.com/cache/280x280/ts/prod...,None,None,False,False,True,False
4,Nồi Cơm Điện Nắp Gài Sunhouse SHD8609 (1.8 Lí...,379.000<sup> ₫</sup>,https://tka.tiki.vn/pixel/pixel?data=obPTD-qKt...,https://salt.tikicdn.com/cache/280x280/media/c...,None,-<!-- -->56<!-- -->%,False,False,True,False
